# Análise dos Acidentes de Trânsito de Belo Horizonte

#### Autores
- Felipe
- Isaac Reyes

## Sumário

- Preparando os dados
  - [Base de dados](#base-de-dados)
  - [Colando um índice de datas](#colando-um-índice-de-datas)
  - [Removendo as colunas que não precisamos](#removendo-as-colunas-que-não-precisamos)
- Questões
  - [2. Faça um gráfico de linhas por ano mês indicando o número de acidentes naquele ano mês](#2-faça-um-gráfico-de-linhas-por-ano-mês-indicando-o-número-de-acidentes-naquele-ano-mês)
  - [3. Repita o gráfico acima por ano apenas](#3-repita-o-gráfico-acima-por-ano-apenas)
  - [4. Faça um gráfico de barras por ano indicandos os tipos de acidentes mais comuns no ano](#4-faça-um-gráfico-de-barras-por-ano-indicandos-os-tipos-de-acidentes-mais-comuns-no-ano)
  - [5. Repita o gráfico acima considerando gráficos fatais e não fatais](#5-repita-o-gráfico-acima-considerando-gráficos-fatais-e-não-fatais)
  - [6. Faça a mesma análise por bairro e por acidentes fatais e não fatais](#6-faça-a-mesma-análise-por-bairro-e-por-acidentes-fatais-e-não-fatais)
  - [7. Plotar Mapa de Belo Horizonte por Tipo de Acidente](#7-plotar-mapa-de-belo-horizonte-por-tipo-de-acidente)
  - [8. Plotar Intervalo de Confiança via Bootstrap do Número de Acidentes por Mês](#8-plotar-intervalo-de-confiança-via-bootstrap-do-número-de-acidentes-por-mês)
  - [9. Vamos brincar de regressão](#9-vamos-brincar-de-regressão)
  - [10. Use o Número de Acidentes por bairro de 2019 para Prever 2022](#10-use-o-número-de-acidentes-por-bairro-de-2019-para-prever-2022)
  - [11. Fazer Análises Adicionais de Interesse](#11-fazer-análises-adicionais-de-interesse)


## Preparando os dados

### Base de Dados

A base de dados a ser analisada consiste em registros de acidentes de trânsito na cidade de Belo Horizonte. A base é separada em vários arquivos csv que podem ser lidos diretamente com babypandas. Para isso, use o código abaixo:

In [ ]:
import babypandas as bpd
import pandas as pd
import glob
import matplotlib.pyplot as plt
import numpy as np

df = bpd.DataFrame()
for f in glob.glob('dados/*.csv'):
    aux = bpd.read_csv(f, sep=';')
    df = df.append(aux)

### Colando um índice de datas

Infelizmente, o BabyPandas não lida muito bem com datas. Por isso, vamos tratar as datas usando pandas. A função pd.to_datetime converte texto em datas, funcionando bem no nosso caso.

In [ ]:
data_correta = pd.to_datetime(
    df.get('data hora_boletim').values
)

Agora basta colocar a data na coluna correta

In [ ]:
df = df.assign(
    data_boletim = data_correta
)

### Removendo as colunas que não precisamos

Por fim, vamos remover todas as colunas desnecessárias e configurar um índice.

In [ ]:
df = df.drop(
    columns=['data hora_boletim',
             'data_inclusao',
             'valor_ups',
             'valor_ups_antiga',
             'data_alteracao_smsa',
             'descricao_ups_antiga']
).sort_values(by='data_boletim').set_index('data_boletim')
df

### 2. Faça um gráfico de linhas por ano mês indicando o número de acidentes naquele ano mês.

In [ ]:
grouped_by_month = df.groupby(['ano', 'mes']).size()
grouped_by_month.plot(kind='line')
plt.ylabel('Número de Acidentes')

### 3. Repita o gráfico acima por ano apenas.

In [ ]:
grouped_by_year = df.groupby('ano').size()
grouped_by_year.plot(kind='line')
plt.ylabel('Número de Acidentes')

### 4. Faça um gráfico de barras por ano indicandos os tipos de acidentes mais comuns no ano.

In [ ]:
import random
import random

years = df.get('ano').unique().tolist()

colors = ['#D354F8', '#595B8E', '#31A8A6', '#F54C50', '#14C50C', '#14B1A6', '#183EAA']

for i in range(len(colors)):
    while i > 0 and colors[i] == colors[i-1]:
        colors[i] = '#' + ''.join(random.choices('0123456789ABCDEF', k=6))

spacing = 1
bar_width = 0.1
num_accident_kinds = df.groupby('tipo_acidente').size()

plt.figure(figsize=(12, 8))
for i, year in enumerate(years):
    accidents_per_year = df[df.get('ano') == year].groupby('tipo_acidente').size()
    ax = accidents_per_year.plot(kind='bar', color=colors[i], position=i * (bar_width + spacing), width=bar_width, label=year, edgecolor='black')

plt.title('Tipo de Acidente mais comum por Ano')
plt.xlabel('Tipo de Acidente')
plt.ylabel('Número de Acidentes')
plt.legend()

plt.show()

### 5. Repita o gráfico acima considerando gráficos fatais e não fatais.

In [ ]:
fatals = df[df.get('indicador_fatalidade') == 'SIM'].groupby('tipo_acidente').size()
non_fatals = df[df.get('indicador_fatalidade') == 'NÃO'].groupby('tipo_acidente').size()

ax = fatals.plot(kind='bar', color='red', position=0, width=0.4, label='Fatais')
non_fatals.plot(kind='bar', color='blue', position=1, width=0.4, ax=ax, label='Não Fatais')

plt.title('Acidentes de Trânsito por Tipo e Gravidade')
plt.xlabel('Tipo de Acidente')
plt.ylabel('Número de Acidentes')
plt.legend()

plt.show()

### 6. Faça a mesma análise por bairro e por acidentes fatais e não fatais

In [ ]:
neighborhood_data = df.groupby('bairro').size()
neighborhood_data
neighborhood_records = {}
for neighborhood in neighborhood_data.index:
    records = df[df.get('bairro') == neighborhood]
    neighborhood_records[neighborhood] = records
neighborhood_records_fatals = {neighborhood: neighborhood_records[neighborhood][neighborhood_records[neighborhood].get('indicador_fatalidade') == 'SIM'] for neighborhood in neighborhood_records}
neighborhood_records_non_fatals = {neighborhood: neighborhood_records[neighborhood][neighborhood_records[neighborhood].get('indicador_fatalidade') == 'NÃO'] for neighborhood in neighborhood_records}

In [ ]:
neighborhood_data.plot(kind="bar", stacked=True)
plt.title('Accidents by Neighborhood')
plt.xlabel('Neighborhood')
plt.ylabel('Number of Accidents')
plt.legend(title='Tipo de Acidente')
plt.show()

### 7. Plotar Mapa de Belo Horizonte por Tipo de Acidente:

- Faça um gráfico de dispersão das latitudes e longitudes. O mesmo deve parecer com o mapa de BH.  
Para entender bem os tipos de acidentes, faça gráficos por tipos diferentes de acidentes.

In [ ]:
df.plot(
  kind='hexbin',
  x='lat',
  y='lon',
  mincnt=1,
  gridsize=75,
  bins='log'
 )
plt.title('Mapa de Belo Horizonte')


Os tipos de acidentes são :

In [ ]:
df.get("desc_tipo_acidente").unique()

Acidentes de queda de pessoa por veículo por latitude e longitude :

In [ ]:
grouped_by_QUEDA_DE_PESSOA_DE_VEICULO = df[(df.get('desc_tipo_acidente') == 'QUEDA DE PESSOA DE VEICULO                        ')]
grouped_by_QUEDA_DE_PESSOA_DE_VEICULO.plot(
  kind='hexbin',
  x='lat',
  y='lon',
  mincnt=1,
  gridsize=75,
  bins='log'
 )
plt.title('Acidentes de queda de pessoa por veículo por latitude e longitude')


Acidentes de atropelamento de pessoa sem vítima fatal por latitude e longitude :

In [ ]:
grouped_by_ATROPELAMENTO_DE_PESSOA_SEM_VITIMA_FATAL = df[(df.get('desc_tipo_acidente') == 'ATROPELAMENTO DE PESSOA SEM VITIMA FATAL          ')]
grouped_by_ATROPELAMENTO_DE_PESSOA_SEM_VITIMA_FATAL.plot(
  kind='hexbin',
  x='lat',
  y='lon',
  mincnt=1,
  gridsize=75,
  bins='log'
 )
plt.title('Acidentes de atropelamento de pessoa sem vítima fatal por latitude e longitude')


Acidentes de abalroamento com vítima por latitude e longitude :

In [ ]:
grouped_by_ABALROAMENTO_COM_VITIMA = df[(df.get('desc_tipo_acidente') == 'ABALROAMENTO COM VITIMA                           ')]
grouped_by_ABALROAMENTO_COM_VITIMA.plot(
  kind='hexbin',
  x='lat',
  y='lon',
  mincnt=1,
  gridsize=75,
  bins='log'
 )
plt.title('Acidentes de abalroamento com vítima por latitude e longitude')

Acidentes de colisão de veículos com vítima por latitude e longitude :

In [ ]:
grouped_by_COLISAO_DE_VEICULOS_COM_VITIMA= df[(df.get('desc_tipo_acidente') == 'COLISAO DE VEICULOS COM VITIMA                    ')]
grouped_by_COLISAO_DE_VEICULOS_COM_VITIMA.plot(
  kind='hexbin',
  x='lat',
  y='lon',
  mincnt=1,
  gridsize=75,
  bins='log'
 )
plt.title('Acidentes de colisão de veículos com vítima por latitude e longitude')

Acidentes de choque mecânico com vítima por latitude e longitude :

In [ ]:
grouped_by_CHOQUE_MECANICO_COM_VITIMA= df[(df.get('desc_tipo_acidente') == 'CHOQUE MECANICO COM VITIMA                        ')]
grouped_by_CHOQUE_MECANICO_COM_VITIMA.plot(
  kind='hexbin',
  x='lat',
  y='lon',
  mincnt=1,
  gridsize=75,
  bins='log'
 )
plt.title('Acidentes de choque mecânico com vítima por latitude e longitude')

Acidentes de capotamento ou tombamento com vítima por latitude e longitude :

In [ ]:
grouped_by_CAPOTAMENTO_ou_TOMBAMENTO_COM_VITIMA= df[(df.get('desc_tipo_acidente') == 'CAPOTAMENTO/TOMBAMENTO COM VITIMA                 ')]
grouped_by_CAPOTAMENTO_ou_TOMBAMENTO_COM_VITIMA.plot(
  kind='hexbin',
  x='lat',
  y='lon',
  mincnt=1,
  gridsize=75,
  bins='log'
 )
plt.title('Acidentes de capotamento ou tombamento com vítima por latitude e longitude')

Acidentes de outros com vítima por latitude e longitude :

In [ ]:
grouped_by_OUTROS_COM_VITIMA= df[(df.get('desc_tipo_acidente') == 'OUTROS COM VITIMA                                 ')]
grouped_by_OUTROS_COM_VITIMA.plot(
  kind='hexbin',
  x='lat',
  y='lon',
  mincnt=1,
  gridsize=75,
  bins='log'
 )
plt.title('Acidentes de outros com vítima por latitude e longitude')

Acidentes de queda e ou vazamento de carga de veículo com vítima por latitude e longitude :

In [ ]:
grouped_by_QUEDA_E_OU_VAZAMENTO_DE_CARGA_DE_VEICULO_COM_VITIMA= df[(df.get('desc_tipo_acidente') == 'QUEDA E/OU VAZAMENTO DE CARGA DE VEICULO C/ VITIMA')]
grouped_by_QUEDA_E_OU_VAZAMENTO_DE_CARGA_DE_VEICULO_COM_VITIMA.plot(
  kind='hexbin',
  x='lat',
  y='lon',
  mincnt=1,
  gridsize=75,
  bins='log'
 )
plt.title('Acidentes de queda e ou vazamento de carga de veículo com vítima por latitude e longitude')

Acidentes de atropelamento de pessoa com vítima fatal por latitude e longitude :

In [ ]:
grouped_by_ATROPELAMENTO_DE_PESSOA_COM_VITIMA_FATAL= df[(df.get('desc_tipo_acidente') == 'ATROPELAMENTO DE PESSOA COM VITIMA FATAL          ')]
grouped_by_ATROPELAMENTO_DE_PESSOA_COM_VITIMA_FATAL.plot(
  kind='hexbin',
  x='lat',
  y='lon',
  mincnt=1,
  gridsize=75,
  bins='log'
 )
plt.title('Acidentes de atropelamento de pessoa com vítima fatal por latitude e longitude')

Acidentes de queda de veículo com vítima por latitude e longitude :

In [ ]:
grouped_by_QUEDA_DE_VEICULO_COM_VITIMA= df[(df.get('desc_tipo_acidente') == 'QUEDA DE VEICULO COM VITIMA                       ')]
grouped_by_QUEDA_DE_VEICULO_COM_VITIMA.plot(
  kind='hexbin',
  x='lat',
  y='lon',
  mincnt=1,
  gridsize=75,
  bins='log'
 )
plt.title('Acidentes de queda de veículo com vítima por latitude e longitude')

Acidentes de atropelamento de animal com vítima por latitude e longitude : 

In [ ]:
grouped_by_ATROPELAMENTO_DE_ANIMAL_COM_VITIMA= df[(df.get('desc_tipo_acidente') == 'ATROPELAMENTO DE ANIMAL COM VITIMA                ')]
grouped_by_ATROPELAMENTO_DE_ANIMAL_COM_VITIMA.plot(
  kind='hexbin',
  x='lat',
  y='lon',
  mincnt=1,
  gridsize=75,
  bins='log'
 )
plt.title('Acidentes de atropelamento de animal com vítima por latitude e longitude')

Acidentes de capotamento ou tombamento sem vítima por latitude e longitude :

In [ ]:
grouped_by_CAPOTAMENTO_ou_TOMBAMENTO_SEM_VITIMA= df[(df.get('desc_tipo_acidente') == 'CAPOTAMENTO/TOMBAMENTO SEM VITIMA                 ')]
grouped_by_CAPOTAMENTO_ou_TOMBAMENTO_SEM_VITIMA.plot(
  kind='hexbin',
  x='lat',
  y='lon',
  mincnt=1,
  gridsize=75,
  bins='log'
 )
plt.title('Acidentes de capotamento ou tombamento sem vítima por latitude e longitude')

### 8. Plotar Intervalo de Confiança via Bootstrap do Número de Acidentes por Mês: 

- Utilize a técnica de bootstrap para calcular intervalos de confiança para o número de acidentes em cada mês. Plote esses intervalos juntamente com a média mensal dos acidentes para visualizar a variabilidade e a confiança das estimativas. 

### 9. Vamos brincar de regressão. 

- Inicialmente, faça um gráfico de dispersão onde o eixo X é o número de acidentes em 2019, 2020 ou 2021. 
- Serão três gráficos. 
- Cada ponto no gráfico vai corresponder a um bairro.

Criando todos os mergs :

In [ ]:
cont = df.groupby(['bairro', 'ano']).size().reset_index()
dados_2019 = cont[cont.get('ano') == 2019]
dados_2020 = cont[cont.get('ano') == 2020]
dados_2021 = cont[cont.get('ano') == 2021]
merge_by_acidentes_2019_e_2020 = dados_2019.merge(
  dados_2020,
  on='bairro')
merge_by_acidentes_2019_e_2021 = dados_2019.merge(
  dados_2021,
  on='bairro')
merge_by_acidentes_2020_e_2021 = dados_2020.merge(
  dados_2021,
  on='bairro')

In [ ]:
merge_by_acidentes_2019_e_2020

In [ ]:
merge_by_acidentes_2019_e_2021

In [ ]:
merge_by_acidentes_2020_e_2021

In [ ]:
merge_by_acidentes_2019_e_2020.plot(kind = "scatter", x = "0_x", y = "0_y" )
plt.xlabel('Número de acidentes por bairro em 2019')
plt.ylabel('Número de acidentes por bairro em 2020')

In [ ]:
merge_by_acidentes_2019_e_2021.plot(kind = "scatter", x = "0_x", y = "0_y" )
plt.xlabel('Número de acidentes por bairro em 2019')
plt.ylabel('Número de acidentes por bairro em 2021')

In [ ]:
merge_by_acidentes_2020_e_2021.plot(kind = "scatter", x = "0_x", y = "0_y" )
plt.xlabel('Número de acidentes por bairro em 2020')
plt.ylabel('Número de acidentes por bairro em 2021')

### 10. Use o Número de Acidentes por bairro de 2019 para Prever 2022:

- Também use os de 2020
- Por fim use os de 2022
- Aplique métodos de previsão (como modelos de regressão) para estimar o número de acidentes em 2023, utilizando os dados passados
- Avalie os erros do modelo e discuta as limitações
- Compare os três modelos. A pandemia teve algum impacto, qual?

Primeiramente, vamos definir uma função que expressa  X ou  Y (colunas de um DataFrame) em unidades padronizadas :

In [ ]:
def standard_units(col):
    return (col - col.mean()) / np.std(col)

Agora uma função que calcula a correlação  R a partir das unidades padronizadas:

In [ ]:
def calculate_r(df, x, y):
    '''Returns the average value of the product of x and y, 
       when both are measured in standard units.'''
    x_su = standard_units(df.get(x))
    y_su = standard_units(df.get(y))
    return (x_su * y_su).mean()

Criando o merge para prever o número de acidentes por bairro de 2022 usando 2019 :  

In [ ]:
dados_2022 = cont[cont.get('ano') == 2022]
merge_by_acidentes_2019_e_2022 = dados_2019.merge(
  dados_2022,
  on='bairro')
merge_by_acidentes_2019_e_2022

Funções para calcular o a e o b no caso geral :

In [ ]:
def slope(df, x, y):
    "Returns the slope of the regression line between columns x and y in df (in original units)."
    r = calculate_r(df, x, y)
    return r * np.std(df.get(y)) / np.std(df.get(x))

def intercept(df, x, y):
    "Returns the intercept of the regression line between columns x and y in df (in original units)."
    return df.get(y).mean() - slope(df, x, y) * df.get(x).mean()

In [ ]:
slope_2019_2022 = slope(merge_by_acidentes_2019_e_2022, "0_x", "0_y")
intercept_2019_2022 = intercept(merge_by_acidentes_2019_e_2022, "0_x", "0_y")

In [ ]:
predict_2022_usando_2019 = merge_by_acidentes_2019_e_2022.assign(num_2022_previsto = slope_2019_2022 * merge_by_acidentes_2019_e_2022.get("0_y") + intercept_2019_2022)
predict_2022_usando_2019

In [ ]:

comparacao_previsto_real_2022_usando_2019 = predict_2022_usando_2019.assign(residual= (predict_2022_usando_2019.get("0_y") - predict_2022_usando_2019.get("num_2022_previsto")))
comparacao_previsto_real_2022_usando_2019

Agora, usando 2020:

In [ ]:
merge_by_acidentes_2020_e_2022 = dados_2020.merge(
  dados_2022,
  on='bairro')
merge_by_acidentes_2020_e_2022

In [ ]:
slope_2020_2022 = slope(merge_by_acidentes_2020_e_2022, "0_x", "0_y")
intercept_2020_2022 = intercept(merge_by_acidentes_2020_e_2022, "0_x", "0_y")

In [ ]:
predict_2022_usando_2020 = merge_by_acidentes_2020_e_2022.assign(num_2022_previsto = slope_2020_2022 * merge_by_acidentes_2020_e_2022.get("0_y") + intercept_2020_2022)
predict_2022_usando_2020

In [ ]:
comparacao_previsto_real_2022_usando_2020 = predict_2022_usando_2020.assign(residual= (predict_2022_usando_2020.get("0_y") - predict_2022_usando_2020.get("num_2022_previsto")))
comparacao_previsto_real_2022_usando_2020

Comparando o erro de previsão dos dois modelos :

In [ ]:
residual_total_usando_2019 = ((comparacao_previsto_real_2022_usando_2019.get("residual") ** 2).sum()) / 291
residual_total_usando_2019

In [ ]:
residual_total_usando_2020 = ((comparacao_previsto_real_2022_usando_2020.get("residual") ** 2).sum()) / 291
residual_total_usando_2020

Assim, concluimos que usar o número de acidentes por bairro de 2019 para prever os 2022 é melhor do que usar os número de acidentes por bairro de 2020.

Agora vamos prever os números de acidentes por bairro de 2023 :

### 11. Fazer Análises Adicionais de Interesse:

- Realize análises adicionais que sejam de seu interesse ou relevância para o projeto. Isso pode incluir a correlação entre diferentes variáveis, a análise de hotspots de acidentes, ou a investigação de fatores contribuintes para a gravidade dos acidentes.

Vamos analisar os hotspost de acidentes por região :

In [ ]:
grouped_by_regiao = df.groupby("desc_regional").size().sort_values(ascending = False)
grouped_by_regiao

Assim concluimos que a região Centro-Sul é a regiao com mais acidentes.
- Agora vamos analisar qual é a região com mais acidentes por quilômetro quadrado,
fazendo uma rápida pesquisa as áreas são : Centro-Sul = 31,53 Pampulha = 47,13 Noroeste = 38,16 Oeste = 33,39 Nordeste = 39,59 Venda Nova = 27,80 Barreiro = 53,51 Leste = 28,52 Norte = 33,21

In [ ]:
acidentes__por_km_centro_sul = 409,1
acidentes__por_km_pampulha = 216,1
acidentes__por_km_noroeste = 236,2
acidentes__por_km_oeste  = 262,4
acidentes__por_km_nordeste = 195,4
acidentes__por_km_venda_nova = 236,7
acidentes__por_km_barreiro = 121,0
acidentes__por_km_leste = 218,0
acidentes__por_km_norte = 166,6

A região Centro-Sul continua sendo a região com mais acidentes por quilômetro quadrado.


Agora vamos fazer a mesma analise por bairro:

In [ ]:
grouped_by_bairro = df.groupby("bairro").size().sort_values(ascending = False)
grouped_by_bairro

Observamos uma grande concentração de acidentes nos bairros, Jardim Montanhês e Centro, caracterizando os moairoes hotspots de acidentes em Belo Horizonte.

Vamos analisar agora os acidentes fatais : 

In [ ]:
acidentes_fatais = df[df.get("indicador_fatalidade") == "SIM"]
acidentes_fatais

In [ ]:
acidentes_fatais.groupby("desc_tempo").size()

In [ ]:
acidentes_fatais_com_tempo_ruim = 66 + 3 + 15
poropocao = acidentes_fatais_com_tempo_ruim / (669 - 140)
poropocao

Considerando apenas os acidentes com o tempo informado, a proporção de acientes fatais com um tempo ruim(chuva ou neblina ou nublado) é de aproximadamente 16%, uma quantidade bem significativa, mostrando a influência do clima nos acidentes de trânsito.